# 2901. Longest Unequal Adjacent Groups Subsequence II

# Medium

You are given a string array words, and an array groups, both arrays having length n.

The hamming distance between two strings of equal length is the number of positions at which the corresponding characters are different.

You need to select the longest subsequence from an array of indices [0, 1, ..., n - 1], such that for the subsequence denoted as [i0, i1, ..., ik-1] having length k, the following holds:

For adjacent indices in the subsequence, their corresponding groups are unequal, i.e., groups[ij] != groups[ij+1], for each j where 0 < j + 1 < k.
words[ij] and words[ij+1] are equal in length, and the hamming distance between them is 1, where 0 < j + 1 < k, for all indices in the subsequence.
Return a string array containing the words corresponding to the indices (in order) in the selected subsequence. If there are multiple answers, return any of them.

> Note: strings in words may be unequal in length.

# Example 1:

```
Input: words = ["bab","dab","cab"], groups = [1,2,2]

Output: ["bab","cab"]

Explanation: A subsequence that can be selected is [0,2].

groups[0] != groups[2]
words[0].length == words[2].length, and the hamming distance between them is 1.
So, a valid answer is [words[0],words[2]] = ["bab","cab"].

Another subsequence that can be selected is [0,1].

groups[0] != groups[1]
words[0].length == words[1].length, and the hamming distance between them is 1.
So, another valid answer is [words[0],words[1]] = ["bab","dab"].

It can be shown that the length of the longest subsequence of indices that satisfies the conditions is 2.
```

# Example 2:

```
Input: words = ["a","b","c","d"], groups = [1,2,3,4]

Output: ["a","b","c","d"]

Explanation: We can select the subsequence [0,1,2,3].

It satisfies both conditions.

Hence, the answer is [words[0],words[1],words[2],words[3]] = ["a","b","c","d"].

It has the longest length among all subsequences of indices that satisfy the conditions.

Hence, it is the only answer.

```

# Constraints:

- 1 <= n == words.length == groups.length <= 1000
- 1 <= words[i].length <= 10
- 1 <= groups[i] <= n
- words consists of distinct strings.
- words[i] consists of lowercase English letters.


In [ ]:
from typing import List

class Solution:
    def getWordsInLongestSubsequence(self, words: List[str], groups: List[int]) -> List[str]:
        num_words = len(words)
        
        # This dictionary will store, for each word at index `i`,
        # a list of indices `k` that `words[i]` can connect to (i.e., `words[k]` can follow `words[i]`)
        possible_next_indices = {}
        
        # This dictionary will store, for each word at index `i`,
        # the actual longest subsequence of words that ends with `words[i]`
        # and satisfies all conditions.
        longest_subsequence_ending_at_index = {}
        
        # This list will store the overall longest valid subsequence found so far.
        overall_longest_subsequence = []

        def calculate_hamming_distance(word1: str, word2: str) -> int:
            """
            Calculates the Hamming distance between two words.
            Returns -1 if lengths are different or distance is > 1.
            """
            if len(word1) != len(word2):
                return -1
            
            differences = 0
            for char_idx in range(len(word1)):
                if word1[char_idx] != word2[char_idx]:
                    differences += 1
                # Early exit if difference already exceeds 1, as we only care about distance == 1
                if differences > 1: # Changed from > 2 to > 1 for efficiency based on problem constraint
                    return -1
            return differences

        # Step 1: Pre-calculate all valid direct connections between words.
        # This builds a graph of valid adjacent word transitions.
        for current_idx in range(num_words):
            possible_next_indices[current_idx] = []
            for next_candidate_idx in range(current_idx + 1, num_words): # Only check future indices for potential 'next' words
                if (groups[current_idx] != groups[next_candidate_idx] and
                    calculate_hamming_distance(words[current_idx], words[next_candidate_idx]) == 1):
                    
                    possible_next_indices[current_idx].append(next_candidate_idx)
        
        # Step 2: Dynamic Programming to find the longest subsequence.
        # We iterate backwards from the last word to the first.
        # For each word, we try to append it to the longest subsequence
        # found from its valid 'next' words.
        for current_idx in range(num_words - 1, -1, -1):
            # Initialize the longest subsequence ending at `current_idx`
            # as just the word itself.
            longest_subsequence_ending_at_index[current_idx] = []

            # Check all words that can validly come after `words[current_idx]`
            for next_idx in possible_next_indices[current_idx]:
                # If the subsequence found starting from `next_idx` is longer
                # than what we currently have for `current_idx` (i.e., just `words[current_idx]`),
                # then update `longest_subsequence_ending_at_index[current_idx]`
                # to include that longer subsequence.
                if len(longest_subsequence_ending_at_index[next_idx]) > len(longest_subsequence_ending_at_index[current_idx]):
                    longest_subsequence_ending_at_index[current_idx] = longest_subsequence_ending_at_index[next_idx]
            
            # Now, prepend `words[current_idx]` to the longest subsequence
            # found from its valid successors.
            longest_subsequence_ending_at_index[current_idx] = [words[current_idx]] + longest_subsequence_ending_at_index[current_idx]
            
            # Keep track of the overall longest subsequence found across all starting points.
            if len(longest_subsequence_ending_at_index[current_idx]) > len(overall_longest_subsequence):
                overall_longest_subsequence = longest_subsequence_ending_at_index[current_idx]

        return overall_longest_subsequence

solution = Solution()

# Test Case 1: Simple valid sequence
words1 = ["cat", "bat", "bet", "bot", "but"]
groups1 = [1, 2, 1, 2, 1]
print(solution.getWordsInLongestSubsequence(words1, groups1))  
# Expected Output: ["cat", "bat", "bot", "but"]

# Test Case 2: Another valid sequence
words2 = ["dog", "dot", "hot", "hog", "log"]
groups2 = [1, 2, 1, 2, 1]
print(solution.getWordsInLongestSubsequence(words2, groups2))  
# Expected Output: ["dog", "dot", "hot", "hog", "log"]
# Edge Case 1: No valid transitions
words3 = ["apple", "banana", "cherry"]
groups3 = [1, 1, 1]
print(solution.getWordsInLongestSubsequence(words3, groups3))  
# Expected Output: ["apple"] (or any single word, since no valid transitions exist)

# Edge Case 2: All words belong to the same group
words4 = ["car", "bar", "bat", "bot"]
groups4 = [1, 1, 1, 1]
print(solution.getWordsInLongestSubsequence(words4, groups4))  
# Expected Output: ["car"] (or any single word, since no valid transitions exist)

# Edge Case 3: Words with different lengths
words5 = ["hello", "hell", "help", "helm"]
groups5 = [1, 2, 1, 2]
print(solution.getWordsInLongestSubsequence(words5, groups5))  
# Expected Output: ["hello"] (since different lengths prevent valid transitions)

# Edge Case 4: Large input size
words6 = ["a" * i for i in range(1, 100)]
groups6 = [i % 2 for i in range(1, 100)]
print(solution.getWordsInLongestSubsequence(words6, groups6))  
# Expected Output: A long sequence following the alternating group pattern